In [53]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import rc, font_manager
ticks_font = font_manager.FontProperties(family='serif', style='normal',
                                         size=24, weight='normal', stretch='normal')
import math

In [136]:
from pymatgen.io.cif import CifParser
from pymatgen.io.vasp import Poscar
from pymatgen.core import Structure

In [137]:
empty_database_raw = pd.read_csv('./bulk_1_all_label_kagome_distortions_multi_gph_expand_20241111.csv')

In [138]:
filled_database_raw = pd.read_csv('./bulk_1_all_label_kagome_distortions_multi_gph_expand_G2_20241111.csv')

In [139]:
All_Cu3Au_database_raw = pd.read_csv('./All_Cu3Au_structure_type_perfect_filled.csv')

In [142]:
classes = []
for i in empty_database_raw.index:
    if type(empty_database_raw['Kagome_group'][i])==str:
        if empty_database_raw['Kagome_group'][i] not in classes:
            classes.append(empty_database_raw['Kagome_group'][i])

In [143]:
classes

['Perfect',
 'Near Perfect',
 'Breathing',
 'Tension-b',
 'Shuffle-a',
 'Twist',
 'Site Distortion',
 'Shuffle-c',
 'Tension-a',
 'Shuffle-b']

In [144]:
empty_classes = []
for i in empty_database_raw.index:
    if type(empty_database_raw['Kagome_group'][i])==str:
        if empty_database_raw['Kagome_group'][i] not in filled_classes:
            filled_classes.append(empty_database_raw['Kagome_group'][i])

In [145]:
all_together_database = pd.DataFrame()

In [146]:
all_together_database['cifs_name'] = np.unique(empty_database_raw['cifs_ids'])

In [147]:
all_together_database['empty'] = np.zeros(len(all_together_database))
all_together_database['filled'] = np.zeros(len(all_together_database))
all_together_database['empty_perfect'] = np.zeros(len(all_together_database))
all_together_database['empty_distort'] = np.zeros(len(all_together_database))
all_together_database['filled_perfect'] = np.zeros(len(all_together_database))
all_together_database['filled_distort'] = np.zeros(len(all_together_database))
all_together_database['sg'] = np.zeros(len(all_together_database))
all_together_database['num_atom'] = np.zeros(len(all_together_database))


In [148]:
# get empty nets first
for i in all_together_database.index:
    cifname = all_together_database['cifs_name'][i] 
    masked_empty_database = empty_database_raw[empty_database_raw['cifs_name']==cifname]
    masked_filled_database = filled_database_raw[filled_database_raw['cifs_name']==cifname]
    masked_All_Cu3Au_database = All_Cu3Au_database_raw[All_Cu3Au_database_raw['cifs_name']==cifname]
    
    num_atom = masked_empty_database['num_atom'].values[0]
    space_group_num = masked_empty_database['space_group_num'].values[0]
    all_together_database['sg'][i] = space_group_num
    all_together_database['num_atom'][i] = num_atom
    
    already_classified_gph = []
    if len(masked_All_Cu3Au_database)!=0:
        all_together_database['filled_perfect'][i]=1
        
        
    if len(masked_filled_database)!=0:
        for j in masked_filled_database.index:
            already_classified_gph.append(masked_filled_database['unique_gph_files'][j])
            
            if masked_filled_database['Kagome_group'][j] in classes:
                if masked_filled_database['Kagome_group'][j]=='Perfect':
                    all_together_database['filled_perfect'][i]=1
                else: 
                    all_together_database['filled_distort'][i]=1
    
    if len(masked_empty_database)!=0:
        for j in masked_empty_database.index:
            if masked_empty_database['Kagome_group'][j] in classes:
                if masked_empty_database['Kagome_group'][j]=='Perfect':
                    all_together_database['empty_perfect'][i]=1
                else: 
                    all_together_database['empty_distort'][i]=1
                    
    if all_together_database['empty_distort'][i]+all_together_database['empty_perfect'][i]>0:
        all_together_database['empty'][i]=1
    if all_together_database['filled_distort'][i]+all_together_database['filled_perfect'][i]>0:
        all_together_database['filled'][i]=1
        
    

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [153]:
all_together_database.to_csv('./full_hex_dataset_kagome_database_class_20241111.csv')